In [1]:
import torch

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
print(DEVICE)

BATCH_SIZE = 224
EPOCH = 30

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

transform_base = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])

train_dataset = ImageFolder(root='../dataset/train_', transform=transform_base)
val_dataset = ImageFolder(root='../dataset/valid_', transform=transform_base)

from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

cuda


C:\Users\kyc_o\anaconda3\envs\dl\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding = 1)
        
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 450)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)
    
model_base = Net().to(DEVICE)
optimizer = optim.Adam(model_base.parameters(), lr=0.001)

In [3]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [4]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy

In [5]:
import time
import copy
 
def train_baseline(model ,train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0  
    best_model_wts = copy.deepcopy(model.state_dict()) 
 
    for epoch in range(1, num_epochs + 1):
        since = time.time()  
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader) 
        val_loss, val_acc = evaluate(model, val_loader)
        
        if val_acc > best_acc: 
            best_acc = val_acc 
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since 
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train. Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))   
        print('valid. Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) 
    model.load_state_dict(best_model_wts)  
    return model
 

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)  	 #(16)
torch.save(base,'baseline.pt')

-------------- epoch 1 ----------------
train. Loss: 5.8366, Accuracy: 50.00%
valid. Loss: 5.8366, Accuracy: 50.00%
Completed in 0m 7s
-------------- epoch 2 ----------------
train. Loss: 5.0580, Accuracy: 50.00%
valid. Loss: 5.0580, Accuracy: 50.00%
Completed in 0m 4s
-------------- epoch 3 ----------------
train. Loss: 3.3770, Accuracy: 50.00%
valid. Loss: 3.3770, Accuracy: 50.00%
Completed in 0m 5s
-------------- epoch 4 ----------------
train. Loss: 2.2621, Accuracy: 50.00%
valid. Loss: 2.2621, Accuracy: 50.00%
Completed in 0m 4s
-------------- epoch 5 ----------------
train. Loss: 1.7648, Accuracy: 50.00%
valid. Loss: 1.7648, Accuracy: 50.00%
Completed in 0m 4s
-------------- epoch 6 ----------------
train. Loss: 1.5494, Accuracy: 50.00%
valid. Loss: 1.5494, Accuracy: 50.00%
Completed in 0m 5s
-------------- epoch 7 ----------------
train. Loss: 1.8458, Accuracy: 33.33%
valid. Loss: 1.8458, Accuracy: 33.33%
Completed in 0m 5s
-------------- epoch 8 ----------------
train. Loss: 2.

In [6]:
for batch_idx, (data, target) in enumerate(train_loader):
        print(batch_idx)
        print(data)
        print(target)

0
tensor([[[[0.5333, 0.5098, 0.4745,  ..., 0.2078, 0.2118, 0.2157],
          [0.5294, 0.5059, 0.4745,  ..., 0.2078, 0.2078, 0.2118],
          [0.5216, 0.5020, 0.4784,  ..., 0.2118, 0.2078, 0.2078],
          ...,
          [0.2824, 0.3804, 0.4353,  ..., 0.1137, 0.1059, 0.1059],
          [0.3098, 0.3569, 0.4431,  ..., 0.1176, 0.1020, 0.1020],
          [0.3529, 0.3569, 0.4431,  ..., 0.1608, 0.1098, 0.1020]],

         [[0.6353, 0.6118, 0.5804,  ..., 0.2157, 0.2196, 0.2235],
          [0.6275, 0.6078, 0.5765,  ..., 0.2157, 0.2157, 0.2196],
          [0.6157, 0.5961, 0.5765,  ..., 0.2196, 0.2157, 0.2157],
          ...,
          [0.3059, 0.4039, 0.4510,  ..., 0.1098, 0.1020, 0.1020],
          [0.3333, 0.3804, 0.4588,  ..., 0.1137, 0.0980, 0.0980],
          [0.3765, 0.3804, 0.4588,  ..., 0.1569, 0.1059, 0.0980]],

         [[0.5373, 0.4941, 0.4314,  ..., 0.2039, 0.2078, 0.2118],
          [0.5373, 0.4980, 0.4392,  ..., 0.2039, 0.2039, 0.2078],
          [0.5412, 0.5020, 0.4510,  ...,

In [ ]:
list(train_loader)['target']